In [1]:
import pandas as pd
import os
from keras.models import Model
from keras.applications import VGG16, ResNet50
from keras.applications.vgg16 import preprocess_input
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle, resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
import tensorflow.keras as keras
from keras import layers
import matplotlib.pyplot as plt
from tabulate import tabulate
import joblib
from sklearn.svm import SVC
from skimage import color, io, transform
import pickle
class_label = ['Non-Default(0)','Default(1)'] # env var

In [2]:
# Load the dataset
df = pd.read_csv('D:\Barath Suresh Docs\PROGRAMMING\MACHINE LEARNING\credit_card_fraud_detection\creditcard.csv')
# Separate the fraud and non-fraud examples
fraud = df[df['Class'] == 1]
non_fraud = df[df['Class'] == 0]
# Random under-sampling of the majority class
non_fraud_downsampled = resample(non_fraud, replace=False, n_samples=len(fraud), random_state=42)
# Combine minority class with downsampled majority class
balanced_df = pd.concat([fraud, non_fraud_downsampled])
# Shuffle the examples
balanced_df = shuffle(balanced_df, random_state=42)
# Print the class distribution of the balanced dataset
print(balanced_df['Class'].value_counts())

0    492
1    492
Name: Class, dtype: int64


In [3]:
# Preprocess the data
X = balanced_df.drop('Class', axis=1).values
y = balanced_df['Class'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)   

In [4]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

SVC(kernel='linear')

In [5]:
weightsvm = np.append(svm.coef_[0][:24], svm.intercept_)
weightsvm = weightsvm.reshape(1, 25)
reshaped_weight_svm = svm.coef_[0][:25].reshape(25,1)
#bias = svm.intercept_[0]

In [6]:
# Normalize the values in the matrix
imgX = (X - np.min(X)) / (np.max(X) - np.min(X))
# Reshape the matrix into a 3D array of grayscale images
imgX = np.reshape(imgX, (imgX.shape[0], imgX.shape[1], 1))
imgX = np.repeat(imgX, 3, axis=2)  # Convert grayscale to RGB
# Optionally, resize the images to a specific size
imgX = np.array([transform.resize(x, (224, 224, 3)) for x in X])
print(imgX.shape)

(984, 224, 224, 3)


In [7]:
# Preprocess the images for use with VGG16
imgX = preprocess_input(imgX)

In [8]:
# Load the pre-trained VGG16 model
base_model =  VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))
# Remove the last layer of the VGG16 model
x = base_model.output
x = GlobalAveragePooling2D()(x)
print(base_model.output.shape)

(None, 7, 7, 512)


In [10]:
# Add an SVM with an RBF kernel as the output layer
x = keras.layers.Dense(1, activation='softmax', weights=[reshaped_weight_svm,svm.intercept_])(x)

ValueError: Layer dense_1 weight shape (512, 1) is not compatible with provided weight shape (25, 1).

In [ ]:
# Create a new model with the modified architecture
model = keras.models.Model(inputs=base_model.input, outputs=x)

In [ ]:
# Freeze the weights of the pre-trained VGG16 layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model with binary crossentropy loss and an Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(imgX, y, test_size=0.2, random_state=42)

In [ ]:
# Train the model on the training set
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1)

Epoch 1/10
23/23 [==============================] - 56s 2s/step - loss: 20.6237 - accuracy: 0.5028 - val_loss: 0.9996 - val_accuracy: 0.4684
Epoch 2/10
23/23 [==============================] - 40s 2s/step - loss: 6.1753 - accuracy: 0.5339 - val_loss: 4.4806 - val_accuracy: 0.4684
Epoch 3/10
23/23 [==============================] - 40s 2s/step - loss: 4.1161 - accuracy: 0.5395 - val_loss: 5.4633 - val_accuracy: 0.5316
Epoch 4/10
23/23 [==============================] - 41s 2s/step - loss: 9.8989 - accuracy: 0.5424 - val_loss: 6.3930 - val_accuracy: 0.4684
Epoch 5/10
23/23 [==============================] - 41s 2s/step - loss: 4.4236 - accuracy: 0.5014 - val_loss: 1.0292 - val_accuracy: 0.5949
Epoch 6/10
23/23 [==============================] - 41s 2s/step - loss: 4.4097 - accuracy: 0.5155 - val_loss: 6.1336 - val_accuracy: 0.5316
Epoch 7/10
23/23 [==============================] - 42s 2s/step - loss: 6.4816 - accuracy: 0.4986 - val_loss: 1.1705 - val_accuracy: 0.5949
Epoch 8/10
23/23 [=

In [ ]:
# Evaluate the model on the testing set
accuracy = model.evaluate(X_test, y_test)[1]
print('Accuracy:', accuracy)

7/7 [==============================] - 10s 1s/step - loss: 15.6601 - accuracy: 0.4365
Accuracy: 0.43654823303222656


In [ ]:
# # Add an SVM kernel layer
# svm = SVC(kernel='rbf')
# x = svm.fit(x, y_train).support_vectors_

In [ ]:
# # Create the new model
# model = Model(inputs=base_model.input, outputs=x)

In [ ]:
# # Freeze the pre-trained layers
# for layer in base_model.layers:
#     layer.trainable = False

In [ ]:
# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# # Train the model
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# # Evaluate the model
# y_pred = model.predict(X_test)
# print(classification_report(y_test, y_pred))